In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [ ]:
img=cv2.imread("/content/X-Ray_image.jpeg")
plt.imshow(img)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
channels=img.shape[2]

In [ ]:
channels

In [ ]:
if(channels==1): # If grayscale
  In=np.asarray(img.flat)
  In
if(channels==3): # If 3-channel rgb
  Lab_img=cv2.cvtColor(img,cv2.COLOR_BGR2LAB)
  L,a,b=cv2.split(Lab_img)
  L1=L.copy()
  In=np.asarray(L.flat)

In [ ]:
plt.imshow(L)

In [ ]:
L.shape

In [ ]:
L.size

In [ ]:
In.shape

In [ ]:
In.size

In [ ]:
In

In [ ]:
#generating a histogram
def gen_histogram(image,bins):
  hist=np.zeros(bins)

  for i in image:
    hist[i]+=1

  return hist



def calc_pdf(hist): #calculating probability density function
  N=np.sum(hist)
  N
  pdf=np.zeros(256)
  for i in range(0,256):
    pdf[i]=hist[i]/N

  return pdf

def check_limit(hist, limit):
    flag=False
    for i in range(256):
        if hist[i] > limit:
          print(i)
          flag=True
          return flag
          break  # Return True if any bin exceeds the limit
    return flag  # Return False if no bin exceeds the limit


def clip(image, clip_limit, hist):
  limit = image.shape[0] * image.shape[1] * clip_limit / 100
  print(limit)
  extrapixels = 0
  l = []
  while check_limit(hist, limit):

        # Find excess pixels and limit bins exceeding the limit
      for i in range(256):
          if hist[i] > limit:
             extrapixels += hist[i] - limit
             hist[i] = limit
             l.append(i)

        # Redistribute excess pixels among other bins
  print(l)
  sharepixels = extrapixels // (256 - len(l))  # Pixels to distribute per bin
  print(sharepixels)
  for i in range(256):
       if i in l:
        hist[i]=limit
       else:
         hist[i] += sharepixels

  return hist.astype('int'), l


def calc_cdf(image): #Calculating cumulative density function
  cdf=np.zeros(256)
  for i in range(0,256):
    if i==256:
       cdf[i]= np.sum(image)
    else:
      for j in range(0,i+1):
        cdf[i]+=image[j]

  return cdf

def new_intensities(image):
  for i in range(0,256):
    image[i]=image[i]*255
  return image

def get_new_image(New_values, Intensity_array):
    for i in range(len(Intensity_array)):
        if 0 <= Intensity_array[i] < len(New_values):
            Intensity_array[i] = New_values[Intensity_array[i]]
    return Intensity_array


In [ ]:
Histogram=gen_histogram(In,256)
plt.plot(Histogram)

In [ ]:
HistInt=Histogram.astype('int')
HistInt

In [ ]:
np.sum(HistInt)

In [ ]:
Clip_Hist,l=clip(L,2,HistInt)

In [ ]:
Clip_Hist

In [ ]:
np.sum(Clip_Hist)

In [ ]:
plt.plot(Clip_Hist)

In [ ]:
PDF=calc_pdf(Clip_Hist)
PDF

In [ ]:
CDF=calc_cdf(PDF)
CDF

In [ ]:
New_pixel=new_intensities(CDF)
New_pixel

In [ ]:
New_pixel_int=New_pixel.astype('int')
New_pixel_int

In [ ]:
Equal_L=get_new_image(New_pixel_int, In)
Equal_L

In [ ]:
Equal_L.shape

In [ ]:
Equal_Hist=gen_histogram(Equal_L,256)
plt.plot(Equal_Hist)

In [ ]:
Equal_Hist.astype('int')

In [ ]:
Equal_L = np.reshape(Equal_L, L.shape)

In [ ]:
plt.imshow(Equal_L, cmap='gray')

In [ ]:
score, _ = ssim(L1, Equal_L, full=True)

In [ ]:
score

In [ ]:
_

In [ ]:
def evaluate_fitness(original_image, processed_image):

    ogn_flat = original_image.flatten()
    pcs_flat = processed_image.flatten()

    mse = np.mean((ogn_flat - pcs_flat) ** 2)
    max_pixel = 255.0

    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [ ]:
PSNR=evaluate_fitness(L1, Equal_L)

In [ ]:
PSNR

In [ ]:
New_img= cv2.merge((Equal_L,a,b))

In [ ]:
plt.imshow(New_img)

In [ ]:
Hist_img= cv2.cvtColor(New_img,cv2.COLOR_LAB2BGR)

In [ ]:
plt.imshow(Hist_img)

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Convert images from BGR to RGB format (assuming OpenCV reads images in BGR format)
original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
enhanced_image = cv2.cvtColor(Hist_img, cv2.COLOR_BGR2RGB)

# Create a figure and axis objects
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Display original image
axes[0].imshow(img)
axes[0].set_title('Original')  # Add label 'Before' to the original image
axes[0].axis('off')

# Display enhanced image
axes[1].imshow(Hist_img)
axes[1].set_title('Enhanced')  # Add label 'After' to the enhanced image
axes[1].axis('off')

# Show the plot
plt.show()

In [ ]:
clahe=cv2.createCLAHE(clipLimit=3.0,tileGridSize=(8,8))
clahe_img=clahe.apply(L)

In [ ]:
New_img2= cv2.merge((clahe_img,a,b))

In [ ]:
CLAHE_img=cv2.cvtColor(New_img2, cv2.COLOR_LAB2BGR)

In [ ]:
plt.imshow(CLAHE_img)

In [ ]:
plt.imshow(img)